In [1]:
from src.backend.retrieval.file_reader import read_csv
from src.backend.retrieval.pipeline import TwoStagePipeline, preprocess_documents
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
df = read_csv('data/processed/all_data_cleaned.csv', doc_column='cleaned_text', label_column='class_label')
X_train, X_test, y_train, y_test = train_test_split(df[0], df[1], test_size=0.2, random_state=42)


In [3]:
raw_texts_train, tokenized_texts_train, embeddings_train = preprocess_documents(
    X_train, model_name="bert-base-uncased", batch_size=64
)

# Preprocess testing data
raw_texts_test, tokenized_texts_test, embeddings_test = preprocess_documents(
    X_test, model_name="bert-base-uncased", batch_size=64
)

Token indices sequence length is longer than the specified maximum sequence length for this model (616 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1149 > 512). Running this sequence through the model will result in indexing errors


In [4]:
pipeline = TwoStagePipeline(raw_texts_train, model_name="bert-base-uncased")


Token indices sequence length is longer than the specified maximum sequence length for this model (616 > 512). Running this sequence through the model will result in indexing errors


In [5]:
results = []

for query, expected_doc in zip(y_test, X_test):
    retrieved_docs = pipeline.run(query, bm25_top_k=5, faiss_top_k=2)
    retrieved_texts = [doc for doc, score in retrieved_docs]
    
    # Check if the expected document is among the retrieved results
    success = expected_doc in retrieved_texts
    results.append(success)

# Calculate accuracy
accuracy = sum(results) / len(results)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 0.00%


In [ ]:
from sklearn.metrics import precision_score, recall_score

# Binary relevance: 1 if the expected_doc is retrieved, 0 otherwise
y_true = [1 if expected_doc in pipeline.run(query, bm25_top_k=5, faiss_top_k=2) else 0
          for query, expected_doc in zip(y_test, X_test)]
y_pred = [1] * len(y_true)  # Assume all predictions are positive (retrieved)

# Calculate Precision and Recall
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

In [ ]:
pipeline = TwoStagePipeline(X_train)

Token indices sequence length is longer than the specified maximum sequence length for this model (916 > 512). Running this sequence through the model will result in indexing errors


[('���: Flooding in', 24.183031), ('���: Flooding in', 24.183031), ('crazy flooding', 19.70793), ('Photos: Severe flooding in Australia', 17.449879), ('flooding kills two.    …', 14.185497)]
